In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.image import AxesImage
from matplotlib import animation, rc
rc('animation', html='jshtml')

%matplotlib inline

In [ ]:
def create_cost_fn(centre_x, centre_y, reg=None,  reg_const=0):
    """
    returns a convex cost function
    """
    if reg is None:
        def cost(x,y):
            return (x - centre_x)**2 + (y - centre_y)**2
    elif reg == 'l1':
        def cost(x,y):
            return (x - centre_x)**2 + (y - centre_y)**2 + reg_const*(abs(x) + abs(y))
    elif reg == 'l2':
        def cost(x,y):
            return (x - centre_x)**2 + (y - centre_y)**2 + reg_const*(x**2 + y**2)
    elif reg == 'max':
        def cost(x,y):
            return (x - centre_x)**2 + (y - centre_y)**2 + reg_const*(x**10 + y**10)**0.1
    
    return cost


def create_cost_inputs_outputs(cost, xmin=-10, xmax=10, ymin=-10, ymax=10):
    """
    returns numpy arrays x,y,z where z = cost(x,y)
    """
    x = np.linspace(xmin, xmax, num=100) + (np.random.random(100)-0.5)*xmax/50
    y = np.linspace(ymin, ymax, num=100) + (np.random.random(100)-0.5)*ymax/50
    x,y = np.meshgrid(x,y, sparse = False)
    z = cost(x,y)
    
    return x,y,z


def plot_contour(x, y, z, levels = np.arange(0,51,2.5)):
    """
    creates a contour plot where z is some function of x and y
    """
    fig, ax = plt.subplots(figsize = (10,10))
    plt.contour(x, y, z,
                levels = levels
               )
    
    return fig, ax


def create_y_l1_coordinate(x, r):
    """
    Return positive y coordinate on l1 ball of radius r and centre 0 with x-coordinate x
    """
    if x >= 0:
        return r - x
    elif x <= 0:
        return r + x

create_y_l1_coordinates = np.vectorize(create_y_l1_coordinate)


def create_y_l2_coordinate(x, r):
    """
    Return positive y coordinate on l2 ball of radius r and centre 0 with x-coordinate x
    """
    return sqrt(r**2 - x**2)

create_y_l2_coordinates = np.vectorize(create_y_l2_coordinate)


def create_ball_boundary(r, norm):
    """
    returns numpy arrays x and y such that
    [(xi,yi) for i in len(x)] traces out the boundary of
    a ball with radius r using the norm provided
    """
    x = np.linspace(-r,r,100)
    
    if norm == 'l1':
        y_pos = create_y_l1_coordinates(x, r)
    elif norm == 'l2':
        y_pos = np.sqrt(r**2 - x**2)
    elif norm == 'max':
        r10 = r**10
        y_pos = (r10 - x**10)**0.1

    y_neg = -y_pos

    x = np.concatenate([x,x[::-1]])
    y = np.concatenate([y_pos, y_neg[::-1]])
    
    return x,y

def plot_ball(x, y, ax):
    """
    shade in the area enclosed by the coordinates [(xi,yi) for i in len(x)]
    """
    ax.fill(x, y, 'b',
            x, y, 'b',
            alpha = 0.2
           )

def minimise_cost(cost, x, y):
    """
    given list of x and y coordinates, determine the coordinate which minimises
    the cost function cost(x,y)
    """
    costs = cost(x,y)
    min_index = np.argmin(costs)
    return x[min_index], y[min_index]


def do_everything(centre_x, centre_y,
                  ball_radius = 4,
                  norm = 'l1'
                 ):
    """
    create convex cost function with min point (centre_x, centre_y)
    create a ball of radius ball_radius with given norm
    find point in this ball that minimizes the cost function
    create plot containing all this information
    """
    cost = create_cost_fn(centre_x, centre_y)
    cost_inputs_x, cost_inputs_y, cost_outputs = create_cost_inputs_outputs(cost)
    fig, ax = plot_contour(cost_inputs_x, cost_inputs_y, cost_outputs)
    
    x_ball, y_ball = create_ball_boundary(ball_radius, norm)
    plot_ball(x_ball, y_ball, ax)
    
    x_min, y_min = minimise_cost(cost, x_ball, y_ball)
    plt.plot(x_min, y_min,
             marker = 'o',
             color = 'red',
             label = 'Minimum value of cost within shaded region'
            )
    plt.legend()
    return fig, ax

In [ ]:
norm = 'l1'

## Without regularsiation, vary cost function, plot contours and ball, plot optimal parameters as red dot

In [ ]:
angles = np.linspace(0, 2*np.pi)
centres_x = 5*np.cos(angles)
centres_y = 5*np.sin(angles)

for i in range(len(angles)):
    fig, ax = do_everything(centres_x[i], centres_y[i], ball_radius = 1, norm = norm)
    plt.savefig(f'{norm}/{i:03}')
    plt.show()

## With regularisation, vary cost function, determine optimal parameters, plot resulting distribution of parameters

In [ ]:
angles = np.random.random(1000)*np.pi*2
radiuses = np.random.random(1000)*5
centres_x = radiuses*np.cos(angles)
centres_y = radiuses*np.sin(angles)

optimal_x_parameters = []
optimal_y_parameters = []

for i in range(len(angles)):
    cost = create_cost_fn(centres_x[i], centres_y[i], reg = norm, reg_const = 4)
    x, y, z = create_cost_inputs_outputs(cost)
    min_indices = np.unravel_index(z.argmin(), z.shape)
    optimal_x_parameters.append(x[min_indices])
    optimal_y_parameters.append(y[min_indices])

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
plt.scatter(x = optimal_x_parameters, y = optimal_y_parameters, alpha= 0.2)
plt.title(f'Distribution of optimal parameters for various cost functions with {norm}-regularisation',
          fontsize = '14'
         )

## Visualising change in contour as you increase reg constant

In [ ]:
for reg_const in range(1,11):
    cost = create_cost_fn(3,5, reg = norm, reg_const = reg_const)
    x, y, z = create_cost_inputs_outputs(cost)
    min_indices = np.unravel_index(z.argmin(), z.shape)
    fig, ax = plot_contour(x, y, z, levels = np.arange(0,101,5))
    plt.plot(x[min_indices], y[min_indices], marker = 'o', color = 'r')
    plt.title(f'Cost with {norm}-regularisation. Reg_const = {reg_const}', fontsize = 18)
    plt.show()